<div style="text-align: right; font-size: 0.8em; color: #666;">
最終更新: 2026-02-17 21:30 (Colab Interactive Forms Edition)
</div>

# 05. LLMベースの軽量マルチエージェント（Executor/Critic）

このノートブックでは、LLM（3Bクラス）を使用して、**「実行者（Executor）」**と**「批評者（Critic）」**の2つの役割を連携させる最小構成のエージェントシステムを体験します。


In [ ]:
#@title 1. 環境セットアップ (編集禁止)
import os
import sys
import torch

repo_path = '/content/llm_lab'
UPDATE_TAG = '20260217_2130'

if os.path.exists(repo_path):
    !rm -rf {repo_path}

!git clone -b ai_agent https://github.com/akio-kobayashi/llm_lab.git {repo_path}
%cd {repo_path}

!pip install -q -U transformers accelerate bitsandbytes

if 'src' not in sys.path:
    sys.path.append(os.path.abspath('src'))

from src.common import load_llm
from src.agent_core import LLMExecutorCriticAgent, RoleConfig

print(f'セットアップ完了 (Tag: {UPDATE_TAG})')


In [ ]:
#@title 2. モデルのロード (編集禁止)
STABLE_MODEL_ID = "llm-jp/llm-jp-3-3.7b-instruct"
model, tokenizer = load_llm(model_id=STABLE_MODEL_ID, use_4bit=False)

def llm_chat(system_prompt: str, user_prompt: str, max_new_tokens: int = 512, temperature: float = 0.2):
    try:
        prompt = f"{system_prompt}\n\n### 指示:\n{user_prompt}\n\n### 応答:\n"
        inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=2048, truncation_side='left')
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        prompt_len = inputs['input_ids'].shape[1]

        with torch.inference_mode():
            gen_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=temperature,
                top_p=0.9,
                use_cache=True,
                pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id,
                repetition_penalty=1.1,
            )
        new_ids = gen_ids[0][prompt_len:]
        return tokenizer.decode(new_ids, skip_special_tokens=True).strip()
    except Exception as e:
        return f'Error: {e}'


## 3. エージェント設定の調整

右側のフォームからパラメータを変更して、エージェントの挙動を調整してみましょう。

In [ ]:
#@title エージェントの「性格」を設定する { run: "refer" }

#@markdown ### Executor (実行担当) の設定
executor_prompt = "\u3042\u306a\u305f\u306f\u512a\u79c0\u306aAI\u30a2\u30b7\u30b9\u30bf\u30f3\u30c8\u3067\u3059\u3002\u8ad6\u7406\u7684\u304b\u3064\u7c21\u6f54\u306b\u56de\u7b54\u3057\u3066\u304f\u3060\u3055\u3044\u3002" #@param {type:"string"}
executor_temp = 0.3 #@param {type:"slider", min:0.1, max:1.0, step:0.1}
executor_tokens = 500 #@param {type:"number"}

#@markdown ### Critic (批評担当) の設定
critic_prompt = "\u3042\u306a\u305f\u306f\u53b3\u3057\u3044\u6279\u8a55\u5bb6\u3067\u3059\u3002\u4e0d\u5099\u3084\u8ad6\u7406\u306e\u98db\u8e8a\u3092\u5bb9\u820e\u306a\u304f\u6307\u6458\u3057\u3066\u304f\u3060\u3055\u3044\u3002" #@param {type:"string"}
critic_temp = 0.1 #@param {type:"slider", min:0.1, max:1.0, step:0.1}
critic_tokens = 400 #@param {type:"number"}

# 設定の適用
executor_config = RoleConfig(
    name="Executor",
    system_prompt=executor_prompt,
    temperature=executor_temp,
    max_new_tokens=executor_tokens
)

critic_config = RoleConfig(
    name="Critic",
    system_prompt=critic_prompt,
    temperature=critic_temp,
    max_new_tokens=critic_tokens
)

agent = LLMExecutorCriticAgent(llm_chat, role_configs=[executor_config, critic_config])
print("エージェントの設定を更新しました。")


## 4. 動作確認

質問を入力して、エージェントを実行してみましょう。

In [ ]:
#@title 質問の実行 { run: "auto" }
query = "Python\u3067\u30ea\u30b9\u30c8\u5185\u306e\u91cd\u8907\u3092\u524a\u9664\u3059\u308b\u30b3\u30fc\u30c9\u3092\u66f8\u3044\u3066\u3002\u305f\u3060\u3057\u3001\u8981\u7d20\u306e\u9806\u756a\u3092\u7dad\u6301\u3059\u308b\u3053\u3068\u3092\u5fd8\u308c\u306a\u3044\u3067\u3002" #@param {type:"string"}

final_answer, full_log, steps = agent.run_pipeline(query)

print('=' * 70)
print('質問:', query)
print('--- エージェント間のやり取りログ ---')
print(full_log)
print('=' * 70)
